## Install Libraries

In [ ]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 6.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

## LLM setup and configuration

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Specify the LLM model
model_name = "microsoft/Phi-3-mini-4k-instruct"

# Configure for GPU usage
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto", # Automatically use available GPU
    torch_dtype=torch.float16, # Can improve performance on some GPUs
    trust_remote_code=True,
)

# Load the tokenizer for the chosen model
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Create a pipeline object for easy text generation with the LLM
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors.index.json:   0%|          | 0.00/16.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Configure LLM generation parameters

In [ ]:
# Parameters to control LLM's response generation
generation_args = {
    "max_new_tokens": 2048,     # Maximum length of the response
    "return_full_text": False, # Only return the generated text
}

## Builing the `query` function

In [ ]:
def query(messages):
  """
  Sends a conversation history to the AI assistant and returns the answer.

  Args:
      messages (list): A list of dictionaries, each with "role" and "content" keys.

  Returns:
      str: The answer from the AI assistant.
  """

  # Send the full conversation history to the LLM
  output = pipe(messages, **generation_args)

  # Extract the response
  return output[0]['generated_text']

## Example usage of the `query` function

In [ ]:
messages = [
    {"role": "system", "content": """As an AI expert in market analysis for the beverage industry, you are tasked with evaluating various products from multiple companies. Conduct an Aspect-Based Sentiment Analysis on each product, focusing on key aspects such as Taste, Carbonation, Aftertaste, Aroma, Ingredients, Packaging, Price, Variety, Availability, Nutritional Content, Customer Service, Consistency and consumer sentiment areas for the product and not singular reviews. Assign a numerical score out of 10 for each aspect and also provide a consumer sentiment description for each aspect, reflecting the sentiment or performance of that aspect for the product.

Additionally, perform a competitor comparison to gauge how each company's product stands in relation to others in the market using a numerical score for aspects as well as a qualitative analysis.

Identify areas for improvement for each company's product, and based on your analysis, provide targeted recommendations for product innovation.

Please present your findings in a structured JSON format, suitable for conversion into a dataframe. The JSON should include an array of objects, each representing a company's product analysis, with nested objects for aspects, company performance, competitor comparison, and areas for improvement.

Here is the structure for the JSON output:

[
  {
    "company": "Specify the company name",
    "product": "Specify the product name",
    "aspects": {
      //specify aspects: {
        //specify sub aspects : Score
        // include other sub aspects as needed
      },
      // specify other aspects in the same format as needed

    "aspects described": {
      //describe aspect sentiment:
      "aspect name": sentiment description based on reviews,
      // add aspects as needed
    },
    "
    "competitor_comparison_described": {
      //Other Company : description of its comparison to our company
      // Include other competitors as needed
    },
    "areas_for_improvement": {
      "Company Name": {
        "aspect": "Suggested area for improvement with how much to improve the metric",
        // Include other aspects as needed
      }
      // Include other companies as needed
    }
    // Include additional products as needed
  }
  // Include additional companies as needed
]
"""},
    {"role": "user", "content": """Perform the given task for the following reviews:
    Here are some reviews about a product from company A:

I've been a loyal fan of this soda for years, primarily because of its distinct taste that I haven't found anywhere else. The balance of sweetness and spice is just right, making it incredibly satisfying, especially on a hot day. However, I've become more health-conscious over time, and the high content of artificial ingredients is starting to concern me. Additionally, the packaging design could be improved. It's often difficult to open, and I feel like they haven't kept up with more eco-friendly trends, which is important to me.
The aftertaste lingers a bit longer than I'd like, but the initial flavor burst is worth it. It's a good value for the price, especially when you catch it on sale.
I'm not a fan of the packaging. It's hard to open sometimes, but the soda itself is delicious. It's my go-to drink for a quick pick-me-up.
It's readily available, which I appreciate. The taste is fantastic, but I'm becoming more aware of the ingredients and their impact on my health.
The taste has been consistent over the years, which is great, but I'm starting to look for drinks with better ingredients.
Great soda with a consistent taste. Wish the packaging was better and the ingredient list shorter.
I appreciate the effort to cut down on calories, but the artificial sweeteners leave a noticeable aftertaste.
It's got a good fizz, decent price, but the aftertaste and ingredients are a letdown. I do like the lower calorie count, though.
The taste is one of a kind, and the lower calorie content is a bonus, but the packaging isn't the easiest to recycle.
I'm torn. The taste is excellent, and it's affordable, but the ingredients list and the aftertaste leave much to be desired.
I appreciate that it has fewer calories than many other sodas on the market, which helps me enjoy it without too much guilt. That said, the aftertaste can be a bit overpowering and tends to linger longer than I'd like.
I'm not a fan of the packaging. It's hard to open sometimes, but the soda itself is delicious. It's my go-to drink for a quick pick-me-up.
This soda has been a staple in my household for its taste and affordability. It's the kind of drink that seems to go well with any meal and any occasion. The carbonation is perfectly dialed in, providing that satisfying fizz without being too aggressive. On the downside, the ingredient list leaves a lot to be desired, with more artificial additives than I'm comfortable with. I've also noticed that the packaging, while distinctive, hasn't changed much over the years and could benefit from a more modern, sustainable approach
I've tried to get behind this soda because of its unique taste, but the more I drink it, the more the artificial sweeteners bother me. There's an aftertaste that lingers far too long, making the overall experience rather unpleasant. On top of that, the ingredients list is a bit concerning with all the additives and preservatives. I'm all for enjoying a good soda, but I think it's important to be mindful of what's inside the can. Unfortunately, this brand just doesn't meet my expectations on either front.
The taste is fantastic, and it has that signature fizz that I love.
I was initially drawn to this soda because of its lower calorie promise, but I must say, I'm quite disappointed. The taste is overshadowed by an unpleasant aftertaste that just doesn't sit well with me. Additionally, the packaging is a letdown. Not only does it feel cheap and fragile, but it's also not environmentally friendly, which is something I'm increasingly concerned about. I had hoped for a better experience, but between the questionable ingredients and the subpar packaging, I think I'll be looking for my fizzy fix elsewhere.
I honestly can't understand the appeal of this soda. The flavor is just off to me—it tastes artificial and nothing like the description. I've tried it cold, over ice, nothing helps. It's a hard pass for me.


Here are some reviews about a competing product from company B

The price is fair, and the taste is consistent, but I'm always left feeling guilty about the sugar content.
The taste is classic and always refreshing, but I'm trying to watch my calorie intake, and this doesn't help. The packaging is sleek and practical, though.
The sweetness is a bit much for me. I enjoy the occasional can, but I'm trying to be more health-conscious.
The clean aftertaste and simple ingredients make this my preferred soda. I just wish they had more low-calorie options.
This soda's taste is consistent, which I appreciate. The high calorie and sugar content, though, prevent me from enjoying it more often.
I've been drinking this soda for years, and the taste has remained consistent. It's always my first choice when I need something fizzy
The classic taste is there, but I'm not impressed with the variety of flavors available. It would be nice to see some new innovations in their lineup.
The fewer ingredients are a plus, but I'm concerned about the sugar and calorie content. The aftertaste is quite good, though.
The variety isn't vast, but the main flavor is so good that I don't mind too much.
The aroma is mild and the ingredients list is reassuringly short, which I appreciate in a beverage
I just can't get behind the taste of this soda. It's too syrupy and sweet for my palate. I've tried it on different occasions, thinking maybe it would grow on me, but it just hasn't. I'll be avoiding this one in the future.
I was unimpressed with the lack of flavor variety and the overly sweet aftertaste. The packaging is decent, but that's not enough to win me over.
The ingredients list is shorter than some, but that doesn't excuse the excessive sweetness and high calorie content. The aftertaste is also unpleasant
It's a standard soda with a typical sugary taste and decent carbonation. The packaging is fine, and the price is in line with other similar options on the market.
I've been drinking this particular soda for as long as I can remember. It's the kind of drink that seems to define what a classic soda should taste like. The consistency in flavor from bottle to bottle is impressive, and the aftertaste is just right – not too sweet, not too bitter. However, the high calorie and sugar content is becoming a bigger issue for me as I try to adopt a healthier lifestyle. While I enjoy the taste and the nostalgia it brings, I'm increasingly looking for alternatives that offer a similar experience without the health drawbacks. The packaging is a positive aspect, though, as it's clear the company has put thought into environmental impact.


Here are some reviews from another competing product from company C:


As someone who enjoys a wide variety of teas, I find this brand to be a standout for its diverse flavor offerings and commitment to using natural ingredients. Each flavor is unique and well-crafted, providing a refreshing alternative to the typical soda. The packaging is not only visually appealing but also practical, making it easy to store and transport. However, I've had some difficulty finding certain flavors in stores, which can be frustrating. The aftertaste is generally pleasant, although some flavors have a slightly bitter finish that I think could be improved upon. Despite these minor issues, it's a brand I regularly recommend to friends and family.
The taste is good, and the packaging is some of the best I've seen. It's a shame it's not more widely available.
The variety keeps me coming back, and the ingredients feel healthier than other options. The aftertaste is a bit strong, though.
The drink is usually easy to find, and the nutritional content is decent for a sweetened beverage.
The drink leaves a bad aftertaste, and the lack of availability is a major downside. It's not something I'll be seeking out again.
The packaging design is modern and appealing, but the drink itself doesn't always deliver on taste. I've also experienced some inconsistency with the product's quality.
While the price and packaging are positives, the taste has been a bit inconsistent lately. I'm also not thrilled about having to hunt for it at different stores.
The drink offers a decent variety, but I find the flavor to be a bit too mild. Additionally, the cans sometimes arrive dented, which is concerning.
I love the commitment to natural ingredients and the variety they offer.
The price is great, but you get what you pay for with a taste that doesn't quite hit the mark and an aftertaste that's less than pleasant. The ingredients are a mixed bag, and the availability is inconsistent, which adds to the frustration.
For the price, you can't beat the quality of this beverage. The taste is consistently good across the different flavors, and while the aftertaste is noticeable, it's actually quite pleasant and not overpowering. The packaging is sturdy and environmentally friendly, which is a huge plus for me.
The diverse range of flavors keeps my taste buds excited, and the natural ingredients make me feel good about my drink choice. Even though it's not always in stock, the hunt makes it feel like a hidden gem. The packaging is not only stylish but also functional, making it easy to enjoy on the go.
The packaging is visually appealing, and the ingredients are top-notch. Availability could be better.
Refreshing taste, but I wish it was easier to find. The variety is good when you can get it.
I really wanted to like this tea, but the taste is just not for me. It's either too bland or too overpowering, depending on the flavor. I've tried several, hoping to find one that suits me, but I've been disappointed every time. I won't be buying this again.

"""},
]
result = query(messages)
print(result)


## Builing the `chat` function

In [ ]:
# def chat():
#   """Enables interactive chat sessions with the AI assistant."""

#   # Initialize the conversation with instructions for the AI assistant
#   conversation_history = [
#       {"role": "system", "content": "You are a helpful digital assistant. Please provide safe, ethical and accurate information to the user."}
#   ]

#   # Main chat loop
#   while True:
#     user_input = input("You: ")

#     # Check if the user wants to exit the chat
#     if user_input.lower() == "exit":
#         break

#     # Add user's message to the conversation history
#     conversation_history.append({"role": "user", "content": user_input})

#     # Get a response from the AI assistant
#     try:
#         response = query(conversation_history)
#         print("\nAssistant: ", response, "\n")

#         # Record the AI assistant's response in the conversation history
#         conversation_history.append({"role": "assistant", "content": response})

#     except Exception as e:
#         print(f"An error occurred: {e}, please try again.")

## Initiating a chat session using the `chat` function

In [ ]:
# chat()